In [3]:
import findspark
findspark.init('/home/ecomog/spark-2.4.3-bin-hadoop2.7')
import pyspark

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import (VectorAssembler, StringIndexer, 
                                VectorIndexer, Normalizer, StandardScaler)
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.ml.classification import RandomForestClassifier,DecisionTreeClassifier
from pyspark.sql import SparkSession

In [2]:
# Start a spark session
spark = SparkSession.builder.appName('finddonors').getOrCreate()

In [11]:
# Load training data
data = spark.read.csv('census.csv', header=True)

In [6]:
# Exploring the dataset
data.count(), len(data.columns)

(45223, 14)

In [12]:
data

DataFrame[age: string, workclass: string, education_level: string, education-num: string, marital-status: string, occupation: string, relationship: string, race: string, sex: string, capital-gain: string, capital-loss: string, hours-per-week: string, native-country: string, income: string]

In [13]:
data.printSchema()

root
 |-- age: string (nullable = true)
 |-- workclass: string (nullable = true)
 |-- education_level: string (nullable = true)
 |-- education-num: string (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital-gain: string (nullable = true)
 |-- capital-loss: string (nullable = true)
 |-- hours-per-week: string (nullable = true)
 |-- native-country: string (nullable = true)
 |-- income: string (nullable = true)



In [14]:
data.head(3)

[Row(age='39', workclass=' State-gov', education_level=' Bachelors', education-num='13.0', marital-status=' Never-married', occupation=' Adm-clerical', relationship=' Not-in-family', race=' White', sex=' Male', capital-gain='2174.0', capital-loss='0.0', hours-per-week='40.0', native-country=' United-States', income='<=50K'),
 Row(age='50', workclass=' Self-emp-not-inc', education_level=' Bachelors', education-num='13.0', marital-status=' Married-civ-spouse', occupation=' Exec-managerial', relationship=' Husband', race=' White', sex=' Male', capital-gain='0.0', capital-loss='0.0', hours-per-week='13.0', native-country=' United-States', income='<=50K'),
 Row(age='38', workclass=' Private', education_level=' HS-grad', education-num='9.0', marital-status=' Divorced', occupation=' Handlers-cleaners', relationship=' Not-in-family', race=' White', sex=' Male', capital-gain='0.0', capital-loss='0.0', hours-per-week='40.0', native-country=' United-States', income='<=50K')]

In [15]:
data.describe().collect()

[Row(summary='count', age='45222', workclass='45222', education_level='45222', education-num='45222', marital-status='45222', occupation='45222', relationship='45222', race='45222', sex='45222', capital-gain='45222', capital-loss='45222', hours-per-week='45222', native-country='45222', income='45222'),
 Row(summary='mean', age='38.547941267524656', workclass=None, education_level=None, education-num='10.118460041572686', marital-status=None, occupation=None, relationship=None, race=None, sex=None, capital-gain='1101.4303436380524', capital-loss='88.595418159303', hours-per-week='40.93801689443191', native-country=None, income=None),
 Row(summary='stddev', age='13.217870219055506', workclass=None, education_level=None, education-num='2.552881195075874', marital-status=None, occupation=None, relationship=None, race=None, sex=None, capital-gain='7506.430083745257', capital-loss='404.9560920589649', hours-per-week='12.007508230033462', native-country=None, income=None),
 Row(summary='min',

In [16]:
data.show(truncate=False)

+---+-----------------+---------------+-------------+----------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
|age|workclass        |education_level|education-num|marital-status        |occupation        |relationship  |race               |sex    |capital-gain|capital-loss|hours-per-week|native-country|income|
+---+-----------------+---------------+-------------+----------------------+------------------+--------------+-------------------+-------+------------+------------+--------------+--------------+------+
|39 | State-gov       | Bachelors     |13.0         | Never-married        | Adm-clerical     | Not-in-family| White             | Male  |2174.0      |0.0         |40.0          | United-States|<=50K |
|50 | Self-emp-not-inc| Bachelors     |13.0         | Married-civ-spouse   | Exec-managerial  | Husband      | White             | Male  |0.0         |0.0         |13.0          | United-State